In [19]:
from hdwallet import BIP141HDWallet
from hdwallet.symbols import RVN
from hdwallet.cryptocurrencies import get_cryptocurrency
from src.rpc import RPC, get_address_utxos
from src.chain_params import burn_rvn
from __data__ import credentials, mnemonic, settings


rpc = RPC(credentials['username'], credentials['password'], 8766)
lang = settings['language']
pword = settings['password']
phrase = mnemonic[-1]

In [6]:
account_index = 0
change_index = 2
address_index = 0
hdwallet = {
    '141': lambda a, b, c, d: BIP141HDWallet(a, get_cryptocurrency(a), f"m/44'/175'/{b}'/{c}/{d}")
}

wallet = hdwallet['141'](RVN, account_index, change_index, address_index)
wallet.from_mnemonic(phrase, lang, pword)
p2pkh = wallet.p2pkh_address()
utxos = get_address_utxos(rpc, [p2pkh, 'RTMuF5inxr45Vz1QWb2eGjMx3B4jedBhXQ'])

In [7]:
from src.encode_img import encrypt_img
from src.rip.utils import gpg, get_pgp_block, gen_key
pword = '12345678'
for key in gpg.list_keys():
    for uid in key['uids']:
        if (p2pkh) in uid:
            print(uid.split(' ')[0])
            print(get_pgp_block(key['keyid']))
            print(encrypt_img('image.png', key['fingerprint'], pword))
        else: gen_key(RVN, p2pkh, pword)

R9ygGHVHQTuKCC2Yh8pZQuq9SXBoBX52nM
-----BEGIN PGP PUBLIC KEY BLOCK-----

mE8EZEeAEhMFK4EEAAoCAwR7KvMjs33JYz+OyKeoGQukb/CQc+iUoSmlyTE+J20j
YlRTuCeLGFrCKAf0YADZQ2hh/7ykXQ0FeOTdDhlKG/vXtFBSOXlnR0hWSFFUdUtD
QzJZaDhwWlF1cTlTWEJvQlg1Mm5NIDxSOXlnR0hWSFFUdUtDQzJZaDhwWlF1cTlT
WEJvQlg1Mm5NQHJ2bi5jb2luPoiTBBMTCAA7FiEEU0q+pu3+OA9Q4UroMNpx+NQO
RN8FAmRHgBICGyMFCwkIBwICIgIGFQoJCAsCBBYCAwECHgcCF4AACgkQMNpx+NQO
RN/+HAEAwYJNvjnDfYguFSP3YVDWJwR92s0/RoZ/z7XLWRu0TH8A/21oKLtJhs6w
6tWfYc0RbU1QWLX9U8tg3a0QjJhhLUNZuFMEZEeAEhIFK4EEAAoCAwQUZG7uLoT1
wC5ubI4BSQzMMi7d+Gr2g2Jfb0Bz8xROF1bJAsXiK8tcipwyVUYrdOwHwM1Empoa
VUHSmxCRNbEpAwEIB4h4BBgTCAAgFiEEU0q+pu3+OA9Q4UroMNpx+NQORN8FAmRH
gBICGwwACgkQMNpx+NQORN/CggEA4HDjpxTgX4yJ9gDb0BQKwlDGFCdJgSNsWj0k
O1yVZJwA/22tAPsbJdpjcpaNfss1FiuiKJw+qm8hrGIsMhZXqDm+
=ABjW
-----END PGP PUBLIC KEY BLOCK-----

-----BEGIN PGP MESSAGE-----

jA0ECQMCmQpbXCylnsH30uoB1mTPNwQ2OIBMqvAVdU9tN1JW5iYxi2NEJHjsKw/k
v6Piy6fnVctOyl5pFAiBUfThpAykeZT2hh98SqtKfzNMDDsOnPKO4hYL2j7poi3I
XgyB4lwDeoG5KxnZBYKzrHX

In [8]:
address = utxos[-1]['address']
txid = utxos[-1]['txid']
vout = utxos[-1]['outputIndex']
script = utxos[-1]['script']
satoshis = utxos[-1]['satoshis']
height = utxos[-1]['height']
print(satoshis)
int(0.14705882 * 1e8)

36184210


14705882

In [9]:
from src.rip.rip_11 import MData11_Recieve
from src.rip.utils import gpg, get_pgp_block
for key in gpg.list_keys():
    kid = key['keyid']
    recipient = MData11_Recieve(p2pkh, get_pgp_block(kid))

In [10]:

from src.ipfs_publish import PublishMData
from src.rvn_rpc import sign_message


asset_name = f'PGP_{recipient.chksum}'
rvn_sig = await sign_message(wallet.wif(), recipient.signed)
published_json = PublishMData(
    'tag', recipient.metadata, recipient.signed, rvn_sig
)
print(asset_name, published_json.cid)

FileExistsError: [Errno 17] File exists: 'src/ipfs_json/1efeeaa4a4c99bbca0bc656752d3a257b06dda5da03f1a986f538c3ca92b7a79.json'

In [11]:
from base64 import b16encode, b16decode
from base58 import b58encode
from ecdsa import SECP256k1, SigningKey
from ecdsa.ecdsa import Public_key, Private_key

from hashlib import new, sha256
encoding = 'ascii'
hex = {
    "bytes": lambda b: b if isinstance(b, bytes) else bytes(b, encoding).hex(),
    "double_sha256": lambda b: sha256(sha256(b if isinstance(b, bytes) else bytes(b, encoding)).hexdigest().encode(encoding)).hexdigest(),
    "ripemd160": lambda b: new('ripemd160', b if isinstance(b, bytes) else bytes(b, encoding)).hexdigest()
}
print((double_sha256 := hex['double_sha256'])('Blue Raspberry'))
# (to_bytes := hex['bytes'])('5')
#
# (ripemd160 := hex['ripemd160'])('v')

c36b4d80a1e1a1e36d09469c02df1442d799fc335eae72f83b12d219862bafa1


In [12]:
def transaction_input(*args): return [{'txid': t, 'vout': v} for t, v in args]

In [13]:
from src.rip.rip_14 import MData14
from src.encode_img import img_file
from src.rvn_rpc import sign_message
icon = img_file('image.png')
name = 'Test'
description = 'This is a test.'
asset_type = 'POINTS'
restrictions = ['Rule 34']
keywords = ['Test', 'Points', 'In-Game Currency']
issuer = 'Robby'
website = 'Testing'
website_url = 'hhtps://www.test.io'
contact_name = 'Bob Robinson'
contact_email = 'bob@gmail.com'
business_name = 'The Business'
street = '1234 Sesame Street'
city = 'City'
state = 'NA'
zipcode = '12345'
country = 'USA'
_address = f'{street}, {city}, {state} {zipcode} {country}'
address = f'{business_name} {_address}' if business_name else _address
sell_price = 0
curreny = 'RVN'
other = {}
attachments = []
recipients = {}
algorithm = 'ECDSA'
ipfs =[]
url = []
mdata14 = MData14(
    icon, name, description, asset_type, restrictions, keywords, 
    issuer, website, website_url, contact_name, contact_email, address, 
    sell_price, curreny, other, attachments, recipients, algorithm, ipfs, url
)
mdata14.metadata

{'asset_data': {'icon': '...',
  'name': 'Test',
  'description': 'This is a test.',
  'asset_type': 'POINTS',
  'restrictions': ['Rule 34'],
  'keywords': ['Test', 'Points', 'In-Game Currency']},
 'admin_data': {'issuer': 'Robby',
  'website': 'Testing',
  'contact_name': 'Bob Robinson',
  'website_url': 'hhtps://www.test.io',
  'contact_email': 'bob@gmail.com',
  'address': 'The Business 1234 Sesame Street, City, NA 12345 USA'},
 'sell_data': {'sell_price': 0, 'curreny': 'RVN'},
 'other_data': {},
 'encryption': {'algorithm': 'ECDSA', 'attachments': [], 'recipients': {}},
 'ipfs_cids': [],
 'url_links': []}

In [14]:
mdata = mdata14.metadata
smdata = mdata14.signed_metadata
signature = await sign_message(wallet.wif(), smdata)

In [15]:
bytes(signature, 'ascii').hex()

'48385679536b37766a45735147714b3569306c385a426d6836346d387372534d724a742b7666774c316c573043626558477a44587665584b535343624f45667a4734316851302f6f70494d632b4e364f344478764f49593d'

In [16]:
from os import listdir
b64images = [img_file(i) for i in listdir('images')]
# for image in b64images:
#     print(str(bytes.fromhex(image), 'ascii'))

In [17]:
wallet.p2pkh_address()

'R9ygGHVHQTuKCC2Yh8pZQuq9SXBoBX52nM'

In [18]:
from src.rvn_rpc import is_valid, get_address_utxos

In [20]:
def issue_asset(*args, **kwargs):
    change_address, change, issuer_address, name, quantity, units, reissuable, has_ipfs, hash = args
    kwargs.update(burn_rvn['main'])
    kwargs.update(
        {
            change_address: change, 
            issuer_address: {
                'issue': {
                    'asset_name': name,
                    'asset_quantity': quantity,
                    'units': units,
                    'reissuable': reissuable,
                    'has_ipfs': has_ipfs,
                    'ipfs_hash': hash
                }
            }
        }
    )
    return kwargs
change_address = ''
amount_of_change = 0
quantity = 90
units = 1
reissuable = 1
has_ipfs = 0
ipfs_hash = ''
issue_asset(
    change_address, amount_of_change, 'asset_address', 'ASSET_NAME', 
    quantity, units, reissuable, has_ipfs, ipfs_hash
)

{'RXissueAssetXXXXXXXXXXXXXXXXXhhZGt': 500,
 '': 0,
 'asset_address': {'issue': {'asset_name': 'ASSET_NAME',
   'asset_quantity': 90,
   'units': 1,
   'reissuable': 1,
   'has_ipfs': 0,
   'ipfs_hash': ''}}}

In [ ]:
def send_to_address(*args):
    to_address, amount, change_address, change = args
    return {to_address: amount, change_address: change}

In [ ]:
# check_address = await is_valid('RTMuF5inxr45Vz1QWb2eGjMx3B4jedBhXQ')
# if check_address['isvalid']:
#     print(check_address)
# else: print(check_address['isvalid'])

In [ ]:
# address_data = get_address_balance(*addresses, show_assets=True)

In [ ]:
# address_balances = {
#     asset['assetName']: 
#     {
#         (b := 'balance'): round(asset[b] // 1e8, 8), 
#         (r := 'received'): round(asset[r] // 1e8, 8)
#     } for asset in address_data
# }
# print(address_balances)
# for asset in address_data:
#     if asset['assetName'] == 'RVN':
#         balance = round(asset['balance'] * SATOSHI, 8)
#         received = round(asset['received'] * SATOSHI, 8)
#         print(f'{balance}\n{received}')